In [1]:
import torch
import torch.nn as nn

from torchvision import datasets, transforms

import numpy as np

In [2]:
def get_mec_data(img_tensor):
    """
    Calculate the information capacity of a dataset with black and white images
    """
    n_img, n_row, n_col = img_tensor.shape

    info = n_img * n_row * n_col    # Assume 1 bit per pixel

    return info

def get_mec_ffnn(layer_ffnn: nn.Sequential):
    """
    Get MEC of last feed forward neural net layer
    """
    mec_layer = []

    for layer in layer_ffnn:
        if isinstance(layer, nn.Linear):
            print("The linear layers are: ")
            print(layer)
            dim_in = layer.in_features
            dim_out = layer.out_features
            print(dim_in, dim_out)
            #if layer.bias:
            if mec_layer == []:
                mec = (dim_in + 1) * dim_out
                mec_layer.append(mec)
            else: 
                #print((dim_in + 1) * dim_out)
                mec = min(dim_in, (dim_in + 1) * dim_out)
                mec_layer.append(mec)
    mec_total = np.sum(mec_layer)

    return mec_total
    

In [3]:
# Load the MNIST dataset
transform = transforms.Compose([transforms.Pad(2),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
img_tensor = trainset.data

get_mec_data(img_tensor)

47040000

In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
for data in trainloader:
    print(data[0].shape)
    break

torch.Size([64, 1, 32, 32])


In [5]:
ffnn = nn.Sequential(
            nn.Linear(512*1*1,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096,10)
        )

get_mec_ffnn(ffnn)

The linear layers are: 
Linear(in_features=512, out_features=4096, bias=True)
512 4096
The linear layers are: 
Linear(in_features=4096, out_features=4096, bias=True)
4096 4096
The linear layers are: 
Linear(in_features=4096, out_features=10, bias=True)
4096 10


2109440

In [6]:
def get_compression(dim_in, dim_out):
    """
    Get compression ratio of convolution layer
    """
    comp_ratio = round(dim_in/dim_out, 2)

    print(f"Compression ratio of convolution layer: {comp_ratio} : 1")
    return comp_ratio


In [7]:
transform = transforms.Compose([transforms.Pad(2),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainset.data.shape

torch.Size([60000, 28, 28])

In [18]:
# Get Generalization then resilience (def 6.4)
trainset
labels = [label for _, label in trainset]

# GPT Code
def calculate_class_probabilities(labels):
    # Count the occurrences of each unique class
    unique_classes, counts = np.unique(labels, return_counts=True)
    
    # Calculate probabilities by dividing counts by the total number of labels
    probabilities = counts / len(labels)
    
    # Combine class labels with their corresponding probabilities
    class_probabilities = dict(zip(unique_classes, probabilities))
    
    return class_probabilities

p_dict = calculate_class_probabilities(labels)
ps = [p_dict[i] for i in p_dict.keys()]

ks = []

[0.09871666666666666,
 0.11236666666666667,
 0.0993,
 0.10218333333333333,
 0.09736666666666667,
 0.09035,
 0.09863333333333334,
 0.10441666666666667,
 0.09751666666666667,
 0.09915]